Tài liệu này mang giấy phép Creative Commons Attribution (CC BY).
(c) Nguyễn Ngọc Sáng, Zhukovsky 12/2018.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

# Tăng tốc Python code với Cython và Numba, vncfd_1D update

 *Để thực hành bài này, các bạn hãy cài ipython hoặc jupyter notebook, cython và numba*
 
Nếu bạn đã thực hành phần 2 CFD với Python hay đang dùng vncfd_1D thì sẽ thấy chương trình chạy rất chậm. Cũng vì vậy mà ở cuối bài 17 đã đề xuất sử dụng sơ đồ ẩn hay tính toán song song để giải hệ phương trình Euler 1D. Ngoài ra chúng ta có thể tăng tốc python code với nhiều cách khác nhau trong đó có sử dụng `cython` và `numba`.

Tại sao python lại chậm như vậy? Có một số lý do như:
- Mỗi đối tượng dù đơn giản nhất đều được lưu trữ trong một cấu trúc dữ liệu phức tạp
- Nhiều thao tác gián tiếp truy cập bộ nhớ
- Kiểm tra kiểu biến

So với ngôn ngữ C, viết code trên Python nhanh, gọn và thuận tiên hơn, tuy nhiên chương trình chạy chậm hơn. Cython là sự kết hợp giữa C và Python: Python code chạy với tốc độ C. Thực chất Cython là Python với kiểu biến C, có thể nói như vậy.

Còn Numba là gì? Đây là trình biên dịch được tối ưu hóa JIT(just-in-time) trên cơ sở LLVM (Low Level Virtual Machine). Với Numba, Python code có thể đạt tốc độ của C hay FORTAN.

Bây giờ ta sẽ so sánh tốc độ Python, Cython, Numba qua hàm tìm áp suất pressure_classic_godunov.

In [31]:
#(khối lượng riêng, vận tốc, áp suất)
Pl = [1.0, 0.75, 1.0]
Pr = [0.125, 0.0, 0.1]

# Python code:

In [49]:
g      = 1.4
gp1    = g+1
gm1    = g-1
gp1d2  = (g+1)/2
gm1d2  = (g-1)/2
gp1d2g = (g+1)/(2*g)
gm1d2g = (g-1)/(2*g)
gm1dgp1 = (g-1)/(g+1)
eps    = 1e-6     #epsilon - sai số 

#phương pháp lặp tìm P
def pressure_classic_godunov_python(Pl, Pr):
    r1, u1, p1 = Pl[0], Pl[1], Pl[2]
    r2, u2, p2 = Pr[0], Pr[1], Pr[2]
    
    #vận tốc âm thanh 
    c1 = (g*p1/r1)**0.5
    c2 = (g*p2/r2)**0.5
    
    #phương pháp lặp 
    P0 = (p1*r2*c2 + p2*r1*c1 + (u1-u2)*r1*c1*r2*c2)/(r1*c1+r2*c2)
    if P0 < eps: P0 = eps
    iterations = 50 # max_iteration 
    while (True):
        P = P0 #áp suất P^{n-1}
        if P >= p1: a1 = (r1*(gp1d2*P + gm1d2*p1))**0.5
        else:
            pp = max(eps, P/p1)
            op = 1. - pp**gm1d2g
            if op>=eps: a1 = gm1d2g*r1*c1*(1. - pp)/op
            else: a1 = r1*c1
        if P >= p2: a2 = (r2*(gp1d2*P + gm1d2*p2))**0.5
        else:
            pp = max(eps, P/p2)
            op = 1. - pp**gm1d2g
            if op>=eps: a2 = gm1d2g*r2*c2*(1. - pp)/op
            else: a2 = r2*c2

        z = P/(p1+p2)
        alpha = gm1/(3*g)*(1. - z)/(z**gp1d2g)/(1. - z**gm1d2g) - 1.
        if alpha < 0.: alpha = 0.
        phi = (a2*p1 + a1*p2 + a1*a2*(u1 - u2))/(a1+a2)

        P0 = (alpha*P + phi)/(1. + alpha)#tính P^n
        iterations -= 1
        if (abs(P0 - P) < eps) or (not iterations): break
    #kết thúc vòng lặp! 
  
    return #P, c1, c2, a1, a2

In [50]:
%timeit pressure_classic_godunov_python(Pl, Pr)

The slowest run took 8.00 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12.2 µs per loop


# Thử nghiệm với Numba:

Để biên dịch một hàm bằng Numba, chúng ta thêm khai báo @jit.

In [60]:
import numba
from numba import jit

@jit (nopython=True)
#phương pháp lặp tìm P
def pressure_classic_godunov_numba(Pl, Pr):
    r1, u1, p1 = Pl[0], Pl[1], Pl[2]
    r2, u2, p2 = Pr[0], Pr[1], Pr[2]
    
    #vận tốc âm thanh 
    c1 = (g*p1/r1)**0.5
    c2 = (g*p2/r2)**0.5
    
    #phương pháp lặp 
    P0 = (p1*r2*c2 + p2*r1*c1 + (u1-u2)*r1*c1*r2*c2)/(r1*c1+r2*c2)
    if P0 < eps: P0 = eps
    iterations = 50 # max_iteration 
    while (True):
        P = P0 #áp suất P^{n-1}
        if P >= p1: a1 = (r1*(gp1d2*P + gm1d2*p1))**0.5
        else:
            pp = max(eps, P/p1)
            op = 1. - pp**gm1d2g
            if op>=eps: a1 = gm1d2g*r1*c1*(1. - pp)/op
            else: a1 = r1*c1
        if P >= p2: a2 = (r2*(gp1d2*P + gm1d2*p2))**0.5
        else:
            pp = max(eps, P/p2)
            op = 1. - pp**gm1d2g
            if op>=eps: a2 = gm1d2g*r2*c2*(1. - pp)/op
            else: a2 = r2*c2

        z = P/(p1+p2)
        alpha = gm1/(3*g)*(1. - z)/(z**gp1d2g)/(1. - z**gm1d2g) - 1.
        if alpha < 0.: alpha = 0.
        phi = (a2*p1 + a1*p2 + a1*a2*(u1 - u2))/(a1+a2)

        P0 = (alpha*P + phi)/(1. + alpha)#tính P^n
        iterations -= 1
        if (abs(P0 - P) < eps) or (not iterations): break
    #kết thúc vòng lặp! 
  
    return #P, c1, c2, a1, a2

In [61]:
%timeit pressure_classic_godunov_numba(Pl, Pr)

The slowest run took 26923.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 11.9 µs per loop


# Cython code:

Một điểm khác biệt với python đó là chúng ta phải thêm khai báo kiểu biến.

In [40]:
%load_ext cython

In [43]:
%%cython
cdef double g      = 1.4
cdef double gp1    = g+1
cdef double gm1    = g-1
cdef double gp1d2  = (g+1)/2
cdef double gm1d2  = (g-1)/2
cdef double gp1d2g = (g+1)/(2*g)
cdef double gm1d2g = (g-1)/(2*g)
cdef double gm1dgp1 = (g-1)/(g+1)
cdef double eps    = 1e-6     #epsilon - sai số 

#Giải bài toán phân rã gián đoạn trong trên từng bề mặt thể tích hữu hạn
def pressure_classic_godunov_cython(Pl, Pr):
    cdef double r1 = Pl[0]
    cdef double u1 = Pl[1]
    cdef double p1 = Pl[2]
    cdef double r2 = Pr[0]
    cdef double u2 = Pr[1]
    cdef double p2 = Pr[2]
    
    #vận tốc âm thanh 
    cdef double c1 = (g*p1/r1)**0.5
    cdef double c2 = (g*p2/r2)**0.5

    #phương pháp lặp tìm P
    cdef double P0 = (p1*r2*c2 + p2*r1*c1 + (u1-u2)*r1*c1*r2*c2)/(r1*c1+r2*c2)
    if P0 < eps: P0 = eps
    cdef double z, alpha, phi
    
    cdef int iterations = 50 # max_iteration 
    while (True):
        P = P0 #áp suất P^{n-1}
        if P >= p1: a1 = (r1*(gp1d2*P + gm1d2*p1))**0.5
        else:
            pp = max(eps, P/p1)
            op = 1. - pp**gm1d2g
            if op>=eps: a1 = gm1d2g*r1*c1*(1. - pp)/op
            else: a1 = r1*c1
        if P >= p2: a2 = (r2*(gp1d2*P + gm1d2*p2))**0.5
        else:
            pp = max(eps, P/p2)
            op = 1. - pp**gm1d2g
            if op>=eps: a2 = gm1d2g*r2*c2*(1. - pp)/op
            else: a2 = r2*c2

        z = P/(p1+p2)
        alpha = gm1/(3*g)*(1. - z)/(z**gp1d2g)/(1. - (z**gm1d2g)) - 1.
        if alpha < 0.: alpha = 0.
        phi = (a2*p1 + a1*p2 + a1*a2*(u1 - u2))/(a1+a2)

        P0 = (alpha*P + phi)/(1. + alpha)#tính P^n
        iterations -= 1
        if (abs(P0 - P) < eps) or (not iterations): break
    #kết thúc vòng lặp!
    return #P, c1, c2, a1, a2

In [44]:
%timeit pressure_classic_godunov_cython(Pl, Pr)

The slowest run took 5.20 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.29 µs per loop


# Kết quả:

Python - 12.2 µs,  Numba - 11.9 µs, Cython - 2.29µs. Numba không hiệu quả như mong muốn. **Cython tăng tốc độ lên hơn 5 lần!**

**Bước tiếp theo** ta kiểm tra tốc độ thông qua chương trình giải hệ phương trình Euler 1D bằng phương pháp Godunov. Ta sẽ sử dụng 3 file [Bai_16_python.ipynb](Bai_16_python.ipynb), [Bai_16_numba.ipynb](Bai_16_python.ipynb), [Bai_16_cython.ipynb](Bai_16_python.ipynb).

Kiểm tra tốc độ tính toán 4 bài test với lưới bằng 81 điểm, CFL = 0.45:

Python: 88.6 ms, 142.0 ms, 103.0 ms, 29.5 ms
Numba : 23.7 ms, 21.9  ms, 17.6  ms, 5.76 ms
Cython: 14.6 ms, 18.7  ms, 13.9  ms, 4.6  ms

Ở đây, numba đã có thấy sự hiệu quả của mình, tuy nhiên vẫn chậm hơn cython. Kết quả sẽ còn khác nữa nếu ta tăng số điểm lưới lên 1001, 5001.

Và còn một điều đặc biệt thể hiện trên hình kết quả bài test2 (và ở các bài test khác) đó là numba làm xuất hiện nhiều dao động, nếu lấy CFL = 0.95 thì nghiệm không hội tụ. 

Điều này tôi không biết giải thích tại sao. Bạn nào biết xin cho lời khuyên về cách khắc phục.

# vncdf_1D update ([link](https://github.com/SangVn/vncfd_1D_update))
Trên cơ sở đó, vncdf_1D được update với phiên bản sử dụng cython, thay đổi chủ yếu nằm trong file `godunov_flux.pyx`. Để so sánh, ta chạy hai file huong_dan_su_dung.ipynb trong hai phiên bản và kiểm tra tốc độ 7 bài test (bài toán riemann lưới 1001 điểm, bài toán shu-osher giữ nguyên).

**Kết quả:**

**Bài toán(phương pháp): Riemann(exact solution, godunov, muscl, weno5), Shu-Osher(muscl, weno5, muscl)** 
              
**Python + Numpy: Riemann(3.25 s, 13.5 s, 45.2 s, 8m 43s), Shu-Osher(11.6 s, 2m 12s, 1m 12 s)**

**Cython + Numpy: Riemann(1.69 s, 1.06 s, 12.0 s, 8m 9s),  Shu-Osher(6.15 s, 2m 2s,  38.1 s)**

Ảnh hưởng lớn nhất của thay đổi này đó là tăng tốc độ tính toán với tái cấu trúc godunov, tiếp đến là muscl, còn đối với weno5 - tốc độ không được cải thiện. Kết quả như vậy là do với weno5 chủ yếu tốn thời gian vào việc tái cấu trúc nghiệm.

Ngoài ra trong phiên bản mới vncfd_1D bổ sung thêm hai phương pháp tính dòng là Rusanov và HLL.